<a href="https://colab.research.google.com/github/amitesh23s/Sentiment_analysis_BERT/blob/main/Twitter_sentiment_analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jp797498e_twitter_entity_sentiment_analysis_path = kagglehub.dataset_download('jp797498e/twitter-entity-sentiment-analysis')

print('Data source import complete.')


100%|██████████| 1.99M/1.99M [00:00<00:00, 90.9MB/s]

Extracting files...
Data source import complete.


In [2]:
import pandas as pd

df = pd.read_csv('/content/twitter_training.csv')
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [3]:
df.columns

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [4]:
df = df.rename(columns={'2401': 'Tweet ID', 'Borderlands': 'Entity', 'Positive': 'Sentiment', 'im getting on borderlands and i will murder you all ,': 'tweet'})

In [5]:
df.head()

,Tweet ID,Entity,Sentiment,tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [6]:
df['tweet'].isna().sum()

686

In [7]:
df['Sentiment'].isna().sum()

0

In [8]:
df.shape

(74681, 4)

In [9]:
df = df.dropna(subset=['tweet'])

In [10]:
df.shape

(73995, 4)

In [11]:
df.Sentiment.value_counts()

,count
Sentiment,
Negative,22358
Positive,20654
Neutral,18108
Irrelevant,12875


In [12]:
labels = {}
unique_labels = df.Sentiment.unique()

for index, unique_label in enumerate(unique_labels):
  labels[unique_label] = float(index)

labels

{'Positive': 0.0, 'Neutral': 1.0, 'Negative': 2.0, 'Irrelevant': 3.0}

In [13]:
df['labels'] = df.Sentiment.replace(labels)

df.head()

<ipython-input-13-f03ff292c47f>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['labels'] = df.Sentiment.replace(labels)
<ipython-input-13-f03ff292c47f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = df.Sentiment.replace(labels)


,Tweet ID,Entity,Sentiment,tweet,labels
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,0.0
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,0.0
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,0.0
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,0.0
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,0.0


In [14]:
df.shape

(73995, 5)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train, test = train_test_split(df, test_size=0.15, random_state=42, stratify=df.labels.values)

In [17]:
print("X Train shape:", train.shape)
print("Y Test shape:", test.shape)

X Train shape: (62895, 5)
Y Test shape: (11100, 5)


In [18]:
def drop_rows(df, label_column, drop_fraction):
    return df.groupby(label_column, group_keys=False).apply(
        lambda x: x.sample(frac=1 - drop_fraction, random_state=42)
    )

train = drop_rows(train, label_column="labels", drop_fraction=0.7)
test = drop_rows(test, label_column="labels", drop_fraction=0.7)

<ipython-input-18-a000e39f72b9>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(label_column, group_keys=False).apply(
<ipython-input-18-a000e39f72b9>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(label_column, group_keys=False).apply(


In [19]:
print("X Train shape:", train.shape)
print("Y Test shape:", test.shape)

X Train shape: (18868, 5)
Y Test shape: (3329, 5)


In [20]:
train.labels.value_counts()

,count
labels,
2.0,5701
0.0,5267
1.0,4617
3.0,3283


In [21]:
test.labels.value_counts()

,count
labels,
2.0,1006
0.0,929
1.0,815
3.0,579


In [22]:
! pip install transformers

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer, AutoTokenizer
import numpy as np

In [24]:
model_name = 'bert-base-uncased'

In [25]:
tokenizer = BertTokenizer.from_pretrained(model_name,
                                          do_lower_case=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [26]:
!pip install transformers

In [27]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [28]:
from datasets import Dataset, DatasetDict

In [29]:
train.tweet.head()

,tweet
6127,@ gur _ karni Hi! Thank you for choosing to ce...
14399,"Wow, such a nice Set from Workshop! . Agree? ..."
2587,"""This will be my year"""
50977,Will someone buy me a star pass on my FIFA mob...
10432,you did well try also fable this time around. ...


In [30]:
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)

print(train_ds)
print(test_ds)

Dataset({
    features: ['Tweet ID', 'Entity', 'Sentiment', 'tweet', 'labels', '__index_level_0__'],
    num_rows: 18868
})
Dataset({
    features: ['Tweet ID', 'Entity', 'Sentiment', 'tweet', 'labels', '__index_level_0__'],
    num_rows: 3329
})


In [31]:
def tok_func(batch):
  return tokenizer(batch["tweet"], padding=True, truncation=True)

In [32]:
train_ds_tokenized = train_ds.map(tok_func, batched=True)
test_ds_tokenized = test_ds.map(tok_func, batched=True)

Map:   0%|          | 0/18868 [00:00<?, ? examples/s]

Map:   0%|          | 0/3329 [00:00<?, ? examples/s]

In [34]:
train_ds_tokenized

Dataset({
    features: ['Tweet ID', 'Entity', 'Sentiment', 'tweet', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18868
})

In [35]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    if predictions.ndim > 1:  # If predictions are logits or probabilities
        predictions = predictions.reshape(-1)  # Take the predicted class

    return {'pearson': corr(predictions, labels)}


In [36]:
bs = 64
epochs = 4
lr = 8e-5

In [37]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=train_ds_tokenized, eval_dataset=test_ds_tokenized,
                  tokenizer=tokenizer, compute_metrics=corr_d)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-38-202ace31ed2e>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=train_ds_tokenized, eval_dataset=test_ds_tokenized,


In [39]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.755469,0.582788
2,0.838500,0.548073,0.722408
3,0.838500,0.496276,0.762039
4,0.282200,0.474978,0.773456


In [40]:
df_val = pd.read_csv('/content/twitter_validation.csv')
df_val.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [41]:
df_val = df_val.rename(columns={'3364': 'Tweet ID', 'Facebook': 'Entity', 'Irrelevant': 'Sentiment', 'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣': 'tweet'})

In [42]:
df_val['labels'] = df_val.Sentiment.replace(labels)
df_val.head()

<ipython-input-42-b20eca6451ed>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_val['labels'] = df_val.Sentiment.replace(labels)


,Tweet ID,Entity,Sentiment,tweet,labels
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,1.0
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,2.0
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",2.0
3,4433,Google,Neutral,Now the President is slapping Americans in the...,1.0
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...,2.0


In [43]:
val_ds = Dataset.from_pandas(df_val)
val_ds

Dataset({
    features: ['Tweet ID', 'Entity', 'Sentiment', 'tweet', 'labels'],
    num_rows: 999
})

In [44]:
val_ds_tokenized = val_ds.map(tok_func, batched=True)
val_ds_tokenized

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Dataset({
    features: ['Tweet ID', 'Entity', 'Sentiment', 'tweet', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 999
})

In [45]:
preds_logits = trainer.predict(val_ds_tokenized).predictions.astype(float)
preds_logits

array([[ 1.08007812e+00],
       [ 2.02343750e+00],
       [ 2.12695312e+00],
       [ 1.10156250e+00],
       [ 2.29101562e+00],
       [-7.69042969e-02],
       [ 9.09667969e-01],
       [ 1.59765625e+00],
       [ 2.14453125e+00],
       [-4.88281250e-02],
       [ 2.13378906e-01],
       [ 2.01171875e+00],
       [ 1.04296875e+00],
       [ 2.21484375e+00],
       [-5.90209961e-02],
       [ 1.28540039e-01],
       [ 2.30468750e+00],
       [-5.67245483e-03],
       [ 1.87695312e+00],
       [ 4.55078125e-01],
       [ 9.03808594e-01],
       [ 3.03710938e+00],
       [-2.08129883e-02],
       [ 1.65527344e+00],
       [ 4.95361328e-01],
       [ 2.03320312e+00],
       [ 1.79296875e+00],
       [ 2.17578125e+00],
       [ 2.13671875e+00],
       [-1.08703613e-01],
       [ 2.37274170e-02],
       [ 2.01953125e+00],
       [-4.61730957e-02],
       [ 2.00976562e+00],
       [ 5.22949219e-01],
       [ 6.02050781e-01],
       [ 2.33984375e+00],
       [-2.78167725e-02],
       [ 1.0

In [46]:
preds_logits = np.clip(preds_logits, 0, 3)

In [48]:
categories = np.array([0.0, 1.0, 2.0, 3.0])
preds = np.array([categories[np.abs(categories - pred).argmin()] for pred in preds_logits.flatten()])

In [51]:
preds.shape

(999,)

In [52]:
preds = preds.astype(float)

In [53]:
true_vals = np.array(val_ds_tokenized['labels'])
print(true_vals.shape)
#print(true_vals)
accuracy = np.mean(preds == true_vals)
print(f"Accuracy: {accuracy * 100:.2f}%")

(999,)
Accuracy: 81.08%
